## Install selenium, webdriver

In [90]:
!pip install selenium
!pip install webdriver-manager
# !pip install WebDriverWait

## 🧑‍💻

### Import

In [22]:
import pandas as pd
import numpy as np
import time
import itertools # ใช้ถอด list ซ้อน list
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from io import StringIO
from selenium.webdriver.support.ui import WebDriverWait

### Scraping

#### Check Data

In [26]:
data_stocks_name = pd.read_csv('Stocks_Name.csv')
data_stocks_name.tail()

,Stocks_Name
918,YONG
919,YUASA
920,ZAA
921,ZEN
922,ZIGA


In [28]:
# Check stock name in the data_stocks_name
def check_stock_exists_insensitive(stock_names):
    lowercase_stock_names = set(data_stocks_name['Stocks_Name'].str.lower())
    if isinstance(stock_names, str):
        stock_names = [stock_names]
    for stock_name in stock_names:
        found = stock_name.lower() in lowercase_stock_names
        print(f"Find stock in df    : {stock_name} ✅" if found else f"No Find stock in df : {stock_name} ❌")

check_stock_exists_insensitive(['aav', 'bbL', 'PTT','peeat'])

Find stock in df    : aav ✅
Find stock in df    : bbL ✅
Find stock in df    : PTT ✅
No Find stock in df : peeat ❌


#### Get url of each sector in the SET

In [12]:
start = time.time()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://stockanalysis.com/quote/bkk/bbl/financials/')

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
# driver.quit()

# # <tr role="row">
# rows = soup.find_all('tr', {'role': 'row'})

# links = []
# for row in rows:
#     # Find inside row that there is div class = "btn-quote d-flex align-items-center w-100"
#     div = row.find('div', class_='btn-quote d-flex align-items-center w-100')
#     if div:
#         anchors = div.find_all('a')
#         for a in anchors:
#             href = a.get('href')
#             if href:
#                 full_url = href if href.startswith('http') else 'https://www.set.or.th' + href
#                 links.append(full_url)
# end = time.time()
# print(f'Time : {end-start:.2f} sec')

# for i, link in enumerate(links):
#     print(f"{i+1}. {link}")

# df_sector = pd.DataFrame(links, columns=['URL'])
# sectors = [
#             "Agro", "Consump", "Financial", "Industrials",
#             "Property & Construction", "Resources", "Services", "Technology" ]

# df_sector['Sector'] = sectors
# df_sector

# df.to_csv('set_links.csv', index=False)

#### Get url of each stock in the sector

In [76]:
links # sectors

['https://www.set.or.th/en/market/index/set/agro',
 'https://www.set.or.th/en/market/index/set/consump',
 'https://www.set.or.th/en/market/index/set/fincial',
 'https://www.set.or.th/en/market/index/set/indus',
 'https://www.set.or.th/en/market/index/set/propcon',
 'https://www.set.or.th/en/market/index/set/resourc',
 'https://www.set.or.th/en/market/index/set/service',
 'https://www.set.or.th/en/market/index/set/tech']

In [78]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

xpath = '/html/body/div[1]/div/div/div[2]/div[2]/div[2]/div/div/div/div[4]'  

all_links = []

for url in links:
    driver.get(url)
    driver.implicitly_wait(np.random.randint(15, 20)) 
    start = time.time()
    
    try:
        container = driver.find_element(By.XPATH, xpath) # Locate the container with stock links 
        a_tags = container.find_elements(By.TAG_NAME, "a") # Find all <a> tags within the container
        stock_urls = [a.get_attribute("href") for a in a_tags if a.get_attribute("href")] # Extract href attribute from each <a> tag
        
        end = time.time()    
        print(f"✅ Sector : {url} — {len(stock_urls)} links found! ⌛ Time : {end-start:.2f} sec")
        for link in stock_urls:
            print(link)
        
        all_links.append(stock_urls)       

    except Exception as e:
        print(f"❌ Failed to extract from {url}: {e}")

driver.quit()

✅ Sector : https://www.set.or.th/en/market/index/set/agro — 70 links found! ⌛ Time : 0.92 sec
https://www.set.or.th/en/market/product/stock/quote/EE/price
https://www.set.or.th/en/market/product/stock/quote/GFPT/price
https://www.set.or.th/en/market/product/stock/quote/LEE/price
https://www.set.or.th/en/market/product/stock/quote/NER/price
https://www.set.or.th/en/market/product/stock/quote/PCE/price
https://www.set.or.th/en/market/product/stock/quote/PPPM/price
https://www.set.or.th/en/market/product/stock/quote/STA/price
https://www.set.or.th/en/market/product/stock/quote/TEGH/price
https://www.set.or.th/en/market/product/stock/quote/TFM/price
https://www.set.or.th/en/market/product/stock/quote/TRUBB/price
https://www.set.or.th/en/market/product/stock/quote/UPOIC/price
https://www.set.or.th/en/market/product/stock/quote/UVAN/price
https://www.set.or.th/en/market/product/stock/quote/VPO/price
https://www.set.or.th/en/market/product/stock/quote/AAI/price
https://www.set.or.th/en/market

In [79]:
len(all_links[0]) + len(all_links[1]) + len(all_links[2]) + len(all_links[3]) + len(all_links[4]) + len(all_links[5]) + len(all_links[6]) + len(all_links[7]) 

699

#### Get financial statement data from all_links

In [20]:
all_links[0]

['https://www.set.or.th/en/market/product/stock/quote/EE/price',
 'https://www.set.or.th/en/market/product/stock/quote/GFPT/price',
 'https://www.set.or.th/en/market/product/stock/quote/LEE/price',
 'https://www.set.or.th/en/market/product/stock/quote/NER/price',
 'https://www.set.or.th/en/market/product/stock/quote/PCE/price',
 'https://www.set.or.th/en/market/product/stock/quote/PPPM/price',
 'https://www.set.or.th/en/market/product/stock/quote/STA/price',
 'https://www.set.or.th/en/market/product/stock/quote/TEGH/price',
 'https://www.set.or.th/en/market/product/stock/quote/TFM/price',
 'https://www.set.or.th/en/market/product/stock/quote/TRUBB/price',
 'https://www.set.or.th/en/market/product/stock/quote/UPOIC/price',
 'https://www.set.or.th/en/market/product/stock/quote/UVAN/price',
 'https://www.set.or.th/en/market/product/stock/quote/VPO/price',
 'https://www.set.or.th/en/market/product/stock/quote/AAI/price',
 'https://www.set.or.th/en/market/product/stock/quote/APURE/price',
 

In [49]:
# ดึง 1 ตัว
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url = "https://www.set.or.th/en/market/product/stock/quote/EE/price"
driver.get(url)

xpath = "/html/body/div[1]/div/div/div[2]/div[2]/div[1]/div/div/div/div/div/div/div[1]/div/div[3]" 

try:
    element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, xpath))
    )
    time.sleep(np.random.randint(4, 7))
    element.click()
except Exception as e:
    print(f"❌ Failed to click: {e}")

web_str = driver.page_source
tables = pd.read_html(StringIO(web_str))

cleaned_tables = []
for i, df in enumerate(tables):
    df_clean = df.dropna()
    cleaned_tables.append(df_clean)
    print(f"\n📊 Table {i+1}")
    print(df_clean)

driver.quit()


📊 Table 1
                           Period as of Y/E 2020 31 Dec 2020  \
1                                Assets              1614.46   
2                           Liabilities               164.09   
3                                Equity              1450.36   
4                       Paid-up Capital              2780.00   
5                               Revenue                 5.44   
6   Profit (Loss) from Other Activities                50.47   
7                            Net Profit                 4.99   
8                            EPS (Baht)                    -   
10                              ROA (%)                 0.34   
11                              ROE (%)                 0.34   
12                Net Profit Margin (%)                91.62   

    Y/E 2021 31 Dec 2021  Y/E 2022 31 Dec 2022  Y/E 2023 31 Dec 2023  \
1                1770.59               1657.59                271.48   
2                  49.73                  6.25                 14.55   
3   

In [51]:
# ดึง 1 ตัว (dic)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url = "https://www.set.or.th/en/market/product/stock/quote/EE/price"
driver.get(url)
xpath = "/html/body/div[1]/div/div/div[2]/div[2]/div[1]/div/div/div/div/div/div/div[1]/div/div[3]" 
try:
    element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, xpath))
    )
    time.sleep(np.random.randint(4, 7))
    element.click()
except Exception as e:
    print(f"❌ Failed to click: {e}")

# Extract stock name from h1
try:
    name_container = driver.find_element(By.CLASS_NAME, "quote-info-left-name-symbols")
    stock_name = name_container.find_element(By.TAG_NAME, "h1").text.strip()
    print(f"📌 Stock Name Found: {stock_name}")
except Exception as e:
    stock_name = "Unknown_Stock"
    print(f"⚠️ Could not extract stock name: {e}")

# Extract and clean tables 
web_str = driver.page_source
tables = pd.read_html(StringIO(web_str))
cleaned_tables = []
for i, df in enumerate(tables):
    df_clean = df.dropna()
    cleaned_tables.append(df_clean)
driver.quit()

# Print result summary
stock_data = {stock_name: cleaned_tables}
print(f"✅ Extracted {len(cleaned_tables)} cleaned tables for {stock_name}")

📌 Stock Name Found: EE
✅ Extracted 2 cleaned tables for EE


In [53]:
stock_data

{'EE': [                           Period as of Y/E 2020 31 Dec 2020  \
  1                                Assets              1614.46   
  2                           Liabilities               164.09   
  3                                Equity              1450.36   
  4                       Paid-up Capital              2780.00   
  5                               Revenue                 5.44   
  6   Profit (Loss) from Other Activities                50.47   
  7                            Net Profit                 4.99   
  8                            EPS (Baht)                    -   
  10                              ROA (%)                 0.34   
  11                              ROE (%)                 0.34   
  12                Net Profit Margin (%)                91.62   
  
      Y/E 2021 31 Dec 2021  Y/E 2022 31 Dec 2022  Y/E 2023 31 Dec 2023  \
  1                1770.59               1657.59                271.48   
  2                  49.73                  6.25   

In [47]:
# ดึงทีเดียวจาก all_links

all_links = ["https://www.set.or.th/en/market/index/set/tech"]

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(all_links[0])
time.sleep(5)

xpath = "/html/body/div[1]/div/div/div[2]/div[2]/div[2]/div/div/div/div[4]/div/div/table/tbody"
container = driver.find_element(By.XPATH, xpath)
a_tags = container.find_elements(By.TAG_NAME, "a")
stock_links = [a.get_attribute("href") for a in a_tags if "/quote/" in a.get_attribute("href")]

stock_financials = {}

if stock_links:
    url = stock_links[0]
    driver.get(url)

    xpath_click = "/html/body/div[1]/div/div/div[2]/div[2]/div[1]/div/div/div/div/div/div/div[1]/div/div[3]"
    element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, xpath_click))
    )
    time.sleep(np.random.randint(4, 7))
    element.click()

    name_container = driver.find_element(By.CLASS_NAME, "quote-info-left-name-symbols")
    stock_name = name_container.find_element(By.TAG_NAME, "h1").text.strip()

    web_str = driver.page_source
    tables = pd.read_html(StringIO(web_str))
    cleaned_tables = [df.dropna() for df in tables]

    stock_financials[stock_name] = cleaned_tables

driver.quit()

for name, dfs in stock_financials.items():
    print(f"{name} — {len(dfs)} tables extracted")

CCET — 2 tables extracted


#### EX.

In [87]:
# import requests
# from bs4 import BeautifulSoup

# url = 'https://www.set.or.th/en/market/index/set/tech'
# response = requests.get(url)

# soup = BeautifulSoup(response.text, "html.parser")
# soup.find(
#     'tr', id="tableSimple-jshadmgvo4-group-row-accordion-children-0-0"
# )

In [89]:
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# url = 'https://www.set.or.th/en/market/index/set/service'
# driver.get(url)
# driver.implicitly_wait(20)

# xpath = '/html/body/div[1]/div/div/div[2]/div[2]/div[2]/div/div/div/div[4]'
# container = driver.find_element(By.XPATH, xpath)

# a_tags = container.find_elements(By.TAG_NAME, "a")

# for a in a_tags:
#     print(a.get_attribute("href"))
    
# print(f'Number of links  : {len(a_tags)}')

#### --